# pystac-client CQL Filtering

This notebook demonstrates the use of pystac-client to use CQL Filtering.

In [1]:
from pystac_client import Client

# set pystac_client logger to DEBUG to see API calls
import logging
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.DEBUG)

In [2]:
# STAC API root URL
URL = 'https://planetarycomputer-staging.microsoft.com/api/stac/v1'

# custom headers
headers = []

cat = Client.open(URL, headers=headers)
cat

DEBUG:pystac_client.stac_api_io:GET https://planetarycomputer-staging.microsoft.com/api/stac/v1 Headers: {'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


<Client id=microsoft-pc>

# Search

Here we perform a search with the `Client.search` function, providing a geometry (`intersects`) a datetime range (`datetime`), and filtering by Item properties (`filter`) using CQL-JSON.

In [3]:
# AOI around Delfzijl, in the north of The Netherlands
geom = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          6.42425537109375,
          53.174765470134616
        ],
        [
          7.344360351562499,
          53.174765470134616
        ],
        [
          7.344360351562499,
          53.67393435835391
        ],
        [
          6.42425537109375,
          53.67393435835391
        ],
        [
          6.42425537109375,
          53.174765470134616
        ]
      ]
    ]
}

# limit sets the # of items per page so we can see multiple pages getting fetched
search = cat.search(
    collections = "aster-l1t",
    intersects = geom,
    datetime = "2000-01-01/2010-12-31",
    max_items = 15,
    limit = 5,
    filter = {
        ""
    }
)

# PySTAC ItemCollection
items = search.get_all_items()

# Dictionary (GeoJSON FeatureCollection)
item_json = items.to_dict()

len(items)

DEBUG:pystac_client.stac_api_io:POST https://planetarycomputer-staging.microsoft.com/api/stac/v1/search Headers: {'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '353', 'Content-Type': 'application/json'} Payload: {"limit": 5, "datetime": "2000-01-01T00:00:00Z/2010-12-31T23:59:59Z", "collections": ["aster-l1t"], "intersects": {"type": "Polygon", "coordinates": [[[6.42425537109375, 53.174765470134616], [7.344360351562499, 53.174765470134616], [7.344360351562499, 53.67393435835391], [6.42425537109375, 53.67393435835391], [6.42425537109375, 53.174765470134616]]]}}
DEBUG:pystac_client.stac_api_io:POST https://planetarycomputer-staging.microsoft.com/api/stac/v1/search Headers: {'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '411', 'Content-Type': 'application/json'} Payload: {"limit": 5, "datetime": "2000-01-01

15

In [ ]:
# note that this will work in JupyterLab, but not in a Jupyter Notebook

import IPython.display
IPython.display.JSON(item_json)

In [ ]:
# this cell can be used in Jupyter Notebook. Use above if using JupyterLab

import json
import uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)


RenderJSON(item_json)